In [79]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import json

HELLO WORK

In [80]:
urls = []
liste = []

for loop in range(1,70) :
    job = "Data&analyst"
    pays = "france"
    type_ = "all"
    url = f"https://www.hellowork.com/searchoffers/getsearchfacets?k={job}&k_autocomplete={job}&l={pays}&ray=all&cod=all&d={type_}&p={loop}&mode=scroll&alert=%20&timestamp=1701361414591"
    urls.append(url)

In [81]:
url

'https://www.hellowork.com/searchoffers/getsearchfacets?k=Data&analyst&k_autocomplete=Data&analyst&l=france&ray=all&cod=all&d=all&p=69&mode=scroll&alert=%20&timestamp=1701361414591'

In [82]:
for url in urls :
    response = requests.get(url)
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
    r = requests.get(url, headers=header)
    json = r.json()


    for i in range(len(json["Results"])):
        try : 
            ligne = {
                'id': json['Results'][i]['Id'],
                'offre': json['Results'][i]["OfferTitle"],
                'url_page' : json['Results'][i]["ResponseUrl"],
                'ville': json['Results'][i]['Localisation'],
                'pays': json['Results'][i]['Localisations'][0]['Label'],
                'entreprise': json['Results'][i]['CompanyName'],
                'contrat': json["Results"][i]['ContractType'],
                'date_publi': json['Results'][i]["PublishDate"],
                'id_plateforme' : 'Hello Work',
                'description': json['Results'][i]['Description'],
                'profil_recherche': json['Results'][i]["Profile"],
                'lat' : json['Results'][i]['AddressGeoloc'][0]['position']['lat'],
                'lon' : json['Results'][i]['AddressGeoloc'][0]['position']['lng'],
                
            }
            liste.append(ligne)
        except :
            pass

df = pd.DataFrame(liste)
# datetime
df['date_publi'] = df['date_publi'].apply(lambda x : x[:10])
df['date_publi'] = pd.to_datetime(df['date_publi'], format = '%Y-%m-%d')

# replace None par NaN
df.replace(to_replace=[None], value = np.nan, inplace=True)

# offres qui contiennent Data Analyst
HW_df = df[(df['offre'].str.contains('Data', case=False, na=False)) & (df['offre'].str.contains('Analyst', case=False, na=False))]
HW_df['description'] = HW_df['description'] + '. ' + HW_df['profil_recherche']
HW_df.drop(columns = 'profil_recherche', inplace = True)
HW_df = HW_df.reset_index(drop=True)

C:\Users\theov\AppData\Local\Temp\ipykernel_18728\3237588029.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HW_df['description'] = HW_df['description'] + '. ' + HW_df['profil_recherche']
C:\Users\theov\AppData\Local\Temp\ipykernel_18728\3237588029.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HW_df.drop(columns = 'profil_recherche', inplace = True)


In [83]:
liste_villes = []
for i in range(len(HW_df['ville'])) :
    liste_villes.append(HW_df['ville'][i].split(' - ')[0])

HW_df['ville'] = liste_villes

liste_villes = []
for i in range(len(HW_df['ville'])) :
    liste_villes.append(''.join([i for i in HW_df['ville'][i] if not i.isdigit()]))
HW_df['ville'] = liste_villes

liste_villes = []
for i in range(len(HW_df['ville'])) :
  if (HW_df['ville'][i] == 'Paris e') or (HW_df['ville'][i] == 'Lyon e') or (HW_df['ville'][i] == "Marseille e") :
    liste_villes.append(HW_df['ville'][i][:-2])
  else :
    liste_villes.append(HW_df['ville'][i])
HW_df['ville'] = liste_villes

liste_villes = []
for i in range(len(HW_df['ville'])) :
  if (HW_df['ville'][i] == 'Paris er') or (HW_df['ville'][i] == 'Lyon er') :
    liste_villes.append(HW_df['ville'][i][:-3])
  else :
    liste_villes.append(HW_df['ville'][i])
HW_df['ville'] = liste_villes

HW_df = HW_df[HW_df['entreprise'].notna()]

In [84]:
# Dictionnaire des coordonnées des villes
villes_france = {
    'Dinard': (48.6333, -2.0667),
    'Créteil': (48.7833, 2.4667),
    'Aix-en-Provence' : (43.5297, 5.4474),
    'Nice' : (43.7102, 7.2620),
    'Paris': (48.8566, 2.3522),
    'Rennes' : (48.1173, 1.6778),
    'Grenoble' : (45.1885, 5.7245),
    'Nanterre' : (48.8924, 2.2153),
    'Neuilly-sur-Seine' : (48.8848, 2.2685),
    '7Th Arrondissement': (48.8575, 2.2945),
    'Issy-Les-Moulineaux': (48.8223, 2.2694),
    'Lille': (50.6292, 3.0573),
    'Boulogne-Billancourt': (48.8397, 2.2394),
    'Guyancourt': (48.7719, 2.0751),
    'Hauts': (48.853, 2.338),
    'Aubervilliers': (48.9161, 2.3833),
    'Argenteuil': (48.9476, 2.2566),
    'Marseille': (43.2965, 5.3698),
    'Annecy': (45.8992, 6.1294),
    'Saint-Malo': (48.6475, -2.014),
    'Bourg-Lès-Valence': (44.9333, 4.8902),
    'Saint-Denis': (48.9356, 2.3592),
    'Pons': (45.5764, -0.5541),
    'Le Mans': (48.0061, 0.1996),
    'Loudéac': (48.1787, -2.7464),
    'Levallois-Perret': (48.8932, 2.2901),
    'Gaillard': (46.1924, 6.2276),
    'Craponne': (45.7481, 4.7277),
    'Bordeaux': (44.8378, -0.5792),
    'Vélizy-Villacoublay': (48.7819, 2.1949),
    'Toulouse': (43.6047, 1.4442),
    'Antony': (48.7535, 2.2975),
    'Cognac': (45.6939, -0.3219),
    'Mayenne': (48.3045, -0.6345),
    'Courbevoie': (48.8978, 2.2594),
    'Bezons': (48.9248, 2.2126),
    'Meudon': (48.8131, 2.2385),
    'Vitrolles' : (43.4586, 5.2497),
    'Wasquehal': (50.6767, 3.1345),
    'Nantes': (47.2184, -1.5536),
    'Aix-En-Provence': (43.5297, 5.4474),
    'Suresnes': (48.8716, 2.2294),
    'Villeurbanne': (45.7712, 4.8901),
    'Lyon': (45.7578, 4.832),
    'Manosque': (43.8292, 5.7821),
    'Neuilly-Sur-Seine': (48.8849, 2.2684),
    'Gentilly': (48.8148, 2.3402),
    "Villeneuve-d'Ascq" : (50.6233, 3.1443),
    'Rennes': (48.1173, -1.6778),
    'Puteaux': (48.8848, 2.239),
    'Schiltigheim': (48.6074, 7.7507),
    'Cholet': (47.059, -0.8789),
    'Biot': (43.6183, 7.0954),
    'Rueil-Malmaison': (48.878, 2.1754),
    'Lestrem': (50.6416, 2.6243),
    'Carros': (43.6571, 7.1951),
    'Île-de-France': (48.8566, 2.3522),
    'Massy': (48.7277, 2.2731),
    'Montreuil': (48.8626, 2.4416),
    'Marquette-lez-Lille': (50.6669, 3.0822),
    'Boves': (49.862, 2.2876),
    'La Chapelle-sur-Erdre': (47.2833, -1.567),
    'Tricastin': (44.384, 4.713),
    'Nancy': (48.6921, 6.1844),
    'Montaigu-Vendée': (46.9863, -1.3062),
    'Mondeville': (49.1854, -0.3212),
    'Ancenis-Saint-Géréon': (47.3667, -1.1776),
    'Montpellier': (43.6116, 3.877),
    'Évry-Courcouronnes': (48.6216, 2.4408),
    'La Défense': (48.8926, 2.2395),
    'Ille-et-Vilaine': (48.1135, -1.6756),
    'Blagnac': (43.6316, 1.3923),
    'Saint-Ouen-sur-Seine': (48.9119, 2.333),
    'Lezennes': (50.6116, 3.1093),
    'Moselle': (49.1193, 6.1727),
    'Beauvais': (49.4289, 2.8224),
    'Niort': (46.3237, -0.464),
    'Quimper': (48.0059, -4.0984),
    'Amiens': (49.8942, 2.3023),
    "Saint-Didier-au-Mont-d'Or": (45.8267, 4.7983),
    'Us': (49.0722, 1.9822),
    'Clermont-Ferrand': (45.7772, 3.0823),
    'Chauray': (46.3708, -0.4612),
    'Cergy': (49.0364, 2.0646),
    'Mérignac': (44.8374, -0.5791),
    'Saint-Nazaire': (47.2704, -2.2137),
    'Saint-Étienne': (45.4397, 4.3878),
    'Hauts-de-Seine': (48.853, 2.338),
    'La Hague': (49.7264, -1.9393),
    'Nord': (50.6326, 3.0586),
    'Clichy': (48.9041, 2.3097),
    'Issy-les-Moulineaux': (48.8223, 2.2694),
    'Cestas': (44.7249, -0.7356),
    "Saint-Ouen-l'Aumône": (49.0486, 2.1206),
    'Bagneux': (48.7987, 2.3136),
    'Cherbourg-en-Cotentin': (49.632, -1.618),
    'Bons-en-Chablais': (46.2858, 6.3873),
    'Plaisir': (48.8199, 1.9495),
    'Gennevilliers': (48.9339, 2.3039),
    'Limoges': (45.8336, 1.2611),
    'Pau': (43.2951, -0.3708),
    'Sainte-Julie': (43.3563, 5.3603),
    'Roissy-en-France': (49.004, 2.5205),
    'Chartres': (48.4439, 1.489),
    'Rennes': (48.1173, -1.6778),
}


# Ajouter les colonnes 'lat' et 'lon' au DataFrame
for position in range(len(HW_df['lat'])) :
    try :
        if (HW_df['lat'][position] == 0.0) and (HW_df['lon'][position] == 0.0) :
            HW_df['lat'][position] = villes_france[HW_df['ville'][position]][0]
            HW_df['lon'][position] = villes_france[HW_df['ville'][position]][1]
        else :
            HW_df['lat'][position] = HW_df['lat'][position]
            HW_df['lon'][position] = HW_df['lon'][position]
    except : 
        pass

C:\Users\theov\AppData\Local\Temp\ipykernel_18728\1801831795.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HW_df['lat'][position] = villes_france[HW_df['ville'][position]][0]
C:\Users\theov\AppData\Local\Temp\ipykernel_18728\1801831795.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HW_df['lon'][position] = villes_france[HW_df['ville'][position]][1]
C:\Users\theov\AppData\Local\Temp\ipykernel_18728\1801831795.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

EFINANCIAL CAREERS

In [85]:
urls = []
for loop in range(1,30) :
  page_size = 50
  p = 20
  url = f"https://job-search-api.efinancialcareers.com/v1/efc/jobs/similar?pageSize={page_size}&filterByCompany=false&jobIDs=P35CoMPCZgq6v1PU&page={loop}"
  urls.append(url)

liste = []

for url in urls :
  r = requests.get(url)
  json = r.json()

  for i in range(len(json["data"])):
    try :
      ligne = {
          'id': json["data"][i]['id'],
          'offre': json["data"][i]["title"],
          'url_page': json['data'][i]['detailsPageUrl'],
          'ville': json['data'][i]['jobLocation']['city'],
          'pays': json['data'][i]['jobLocation']['country'],
          'entreprise': json['data'][i]['companyName'],
          'contrat': json['data'][i]["employmentType"],
          'date_publi' : json['data'][i]["postedDate"],
          'id_plateforme':'efinancialcareers'
      }
      liste.append(ligne)
    except :
      pass

df_efinancialcareers = pd.DataFrame(liste)

url_mere = 'https://www.efinancialcareers.fr'
df_efinancialcareers['url_page'] = df_efinancialcareers['url_page'].apply(lambda x: url_mere + x if pd.notna(x) else x)

df_efinancialcareers = df_efinancialcareers[(df_efinancialcareers['offre'].str.contains('Data')) & (df_efinancialcareers['offre'].str.contains('Analyst'))]

df_efinancialcareers['date_publi'] = pd.to_datetime(df_efinancialcareers['date_publi'])
df_efinancialcareers['date_publi'] = df_efinancialcareers['date_publi'].dt.strftime('%Y-%m-%d')

liste_description = []
for url in df_efinancialcareers['url_page'] :
  page = requests.get(url)
  html = page.content
  soup = BeautifulSoup(page.text, 'html')
  description = soup.find_all(class_ = "inner-content ng-star-inserted")[0].text
  liste_description.append(description)

df_efinancialcareers['description'] = liste_description
df_efinancialcareers['date_publi'] = df_efinancialcareers['date_publi'].apply(lambda x : x[:10])
df_efinancialcareers['date_publi'] = pd.to_datetime(df_efinancialcareers['date_publi'], format = '%Y-%m-%d')

In [86]:
# Ajouter les colonnes 'lat' et 'lon' au DataFrame
df_efinancialcareers['lat'] = df_efinancialcareers['ville'].apply(lambda x: villes_france.get(x, (0, 0))[0])
df_efinancialcareers['lon'] = df_efinancialcareers['ville'].apply(lambda x: villes_france.get(x, (0, 0))[1])

Talentify

In [87]:
nbr = 1000
url = f"https://api.talentify.io/search/jobs?query=Data+Analyst&country=FR&totalPageItems={nbr}"

response = requests.get(url)
json = response.json()

liste = []
for i in range(len(json["items"])):
    ligne = {
        'entreprise' : json['items'][i]['companyName'],
        'offre' : json['items'][i]['title'],
        'description' : json['items'][i]['jobSummary'],
        'ville' : json['items'][i]['location'],
        'url_page' : json['items'][i]['jobUrl'],
        'date_publi' : json['items'][i]['publishedAt'],
        'contrat' : json['items'][i]['jobType']
}
    liste.append(ligne)
df = pd.DataFrame(liste)

df.drop_duplicates(keep = 'first', inplace = True)
listes_urls = df['url_page'].values

liste_description = []
liste_ref = []

for url in listes_urls :
  page = requests.get(url)
  html = page.content
  soup = BeautifulSoup(page.text, 'html')
  description = soup.find_all(class_ = 'OpeningInfo__descriptionText')[0].text
  id = soup.find_all(class_ = 'OpeningInfo__referenceNumberText')[0].text[7:]
  liste_ref.append(id)
  liste_description.append(description)

df_2 = pd.DataFrame({'url_page' : listes_urls, 'description' : liste_description, 'id' : liste_ref })
df_2.drop_duplicates(keep = 'first', inplace = True)
df_talentify = df.merge(df_2, how='inner', left_on = 'url_page', right_on = 'url_page')
df_talentify.drop(columns = 'description_x', inplace = True)
df_talentify.rename(columns ={'description_y' : 'description'},inplace = True)
df_talentify.drop_duplicates(keep="first", inplace = True)
df_talentify['id_plateforme'] = 'talentify'

df_talentify['date_publi'] = pd.to_datetime(df_talentify['date_publi'])
df_talentify['date_publi'] = df_talentify['date_publi'].dt.strftime('%Y-%m-%d')

# Villes et Pays
df_talentify = df_talentify.reset_index()
liste_villes = []
for i in range(len(df_talentify['ville'])) :
  liste_villes.append(df_talentify['ville'][i].split(',')[0])

df_talentify['ville'] = liste_villes
df_talentify['pays'] = 'France'

df_talentify = df_talentify[['id', 'offre', 'url_page', 'ville', 'pays', 'entreprise', 'contrat', 'date_publi', 'id_plateforme', 'description']]

df_talentify['date_publi'] = df_talentify['date_publi'].apply(lambda x : x[:10])
df_talentify['date_publi'] = pd.to_datetime(df_talentify['date_publi'], format = '%Y-%m-%d')

In [88]:
# Ajouter les colonnes 'lat' et 'lon' au DataFrame
df_talentify['lat'] = df_talentify['ville'].apply(lambda x: villes_france.get(x, (0, 0))[0])
df_talentify['lon'] = df_talentify['ville'].apply(lambda x: villes_france.get(x, (0, 0))[1])

POLE EMPLOI

In [89]:
token_url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=%2Fpartenaire"
client_id = "PAR_datajobproject_35547f000739b186dc3a5e72ddb87345a1be98c60455598197bbce3a5177b5ce"
client_secret = "2fe5feaaf457d447c94acfead50e58b9960177d0b4445a50156ed463233dc9b3"

payload = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "o2dsoffre"
}

response = requests.post(token_url, data=payload)

if response.status_code == 200:
    access_token = response.json().get("access_token")
    print(f"Access Token: {access_token}")

else:
    print(f"Error: {response.status_code} - {response.text}")

Access Token: 57JWJHkBHe1Po1ZQWemUUhY82Po


In [90]:
import json

In [91]:
url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=partenaire"

Leclient_id = "PAR_datajobproject_35547f000739b186dc3a5e72ddb87345a1be98c60455598197bbce3a5177b5ce"
Leclient_secret = "2fe5feaaf457d447c94acfead50e58b9960177d0b4445a50156ed463233dc9b3"

payload='client_id=' + Leclient_id + '&client_secret=' + Leclient_secret + '&grant_type=client_credentials&scope=api_offresdemploiv2 o2dsoffre'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    data = response.json()
else:
    pass
tk = json.loads(response.text)['access_token']
url = "https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search?motsCles=data%20analyst"

payload={}
headers = {
  'Authorization': 'Bearer '+ str(tk),
  'Content-Type': 'application/json'

}

response = requests.request("GET", url, headers=headers, data=payload)

# Vérifiez si la requête a réussi
if response.status_code == 200 or response.status_code == 206:
    print(response.text)
else:
    # Affichez un message d'erreur si la requête n'a pas réussi
    pass

{"resultats":[{"id":"169DBVQ","intitule":"Data analyst","description":"Au sein de la Direction Commerciale de BAT France et rattaché au Responsable du Déploiement de la Stratégie Commerciale, vous aurez pour principales responsabilités :\n \tSuivre quotidiennement les KPI's et R/O des cycles d'activité commerciale \n \tAssurer un communication régulières auprès des équipes des ventes terrain\n \tSuivre les plans d'actions et proposer des axes d'amélioration\n \tCalculer les objectifs et les formaliser auprès de la force de vente\n \tAnalyser les performances tant sur le plan qualitatif que quantitatif\n\nExpérience requise\n \tVous justifiez d'au moins une première expérience réussie (stages/alternance inclus) sur un poste en dans le domaine Trade-Marketing, idéalement dans le secteur des FMCG.\n\nCompétences requises\n\n \tVous êtes rigoureux, autonome et avez un fort esprit d'équipe \n \tVous avez un excellent sens relationnel et la capacité à prendre des initiatives \n \tVous faites

In [92]:
data = response.json()
liste =[]
for i in range(len(data['resultats'])):

      url_page = data['resultats'][i]['origineOffre']['urlOrigine']

      ligne = {
          'id': data['resultats'][i]['id'],
          'offre': data['resultats'][i]['intitule'],
          'url_page' : url_page,
          'ville': data['resultats'][i]['lieuTravail']['libelle'],
          'pays' : 'France',
          'entreprise': data['resultats'][i]['entreprise'].get('nom', None),
          'contrat': data['resultats'][i].get('typeContrat', None),
          'date_publi': data['resultats'][i]['dateCreation'],
          'id_plateforme' : 'Pole Emploi',
          'description': data['resultats'][i]['description'],
          'lat': data['resultats'][i]['lieuTravail'].get('latitude', None),
          'lon': data['resultats'][i]['lieuTravail'].get('longitude', None),
      }
      liste.append(ligne)

df_pe = pd.DataFrame(liste)

In [93]:
df_pe['date_publi'] = df_pe['date_publi'].apply(lambda x : x[:10])
df_pe['date_publi'] = pd.to_datetime(df_pe['date_publi'], format = '%Y-%m-%d')

In [94]:
import re

text_to_replace = [ '\\', '/', '\n', '*', '#', '✨', '***', '\n1', '\n*', '\n+', '\xa0  *','\xa0\xa0', '\xa0']
escaped_text_to_replace = [re.escape(pattern) for pattern in text_to_replace]

for pattern in escaped_text_to_replace:
    df_pe['description'] = df_pe['description'].str.replace(pattern, ' ')

df_pe['ville'] = df_pe['ville'].apply(lambda x: re.search(r'\d+ - (.+)', x).group(1) if re.search(r'\d+ - (.+)', x) else re.search(r'[A-Z].*', x).group() if re.search(r'[A-Z].*', x) else x)
df_pe['ville'] = df_pe['ville'].apply(lambda x: re.sub(r'\d+', '', x) if re.search(r'\d+', x) else x)

df_pe['ville'] = df_pe['ville'].str.lower().str.title()


df_final

In [95]:
df = pd.concat([df_efinancialcareers, df_talentify, HW_df, df_pe])
df = df.sort_values(by = 'date_publi', ascending = False)
df = df[df['date_publi'] > '2023-10-01']
df = df.reset_index(drop = True)

In [96]:
def preprocess_text(text):
    # Convertir le texte en minuscules
    text = text.lower()

    # Supprimer la ponctuation
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))

    # Tokenizer le texte
    tokens = word_tokenize(text, language='french')

    # Enlever les mots vides (stopwords)
    stop_words = set(stopwords.words('french'))
    tokens = [word for word in tokens if word not in stop_words]

    # Appliquer la racinisation (stemming)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

In [97]:
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
from nltk import FreqDist
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

In [98]:
df['ville'] = df['ville'].str.strip()

In [99]:
df['ville'] = df['ville'].replace('Aix En Provence', 'Aix-En-Provence')
df['ville'] = df['ville'].replace('Aix-en-Provence', 'Aix-En-Provence')

In [100]:
df['ville'] = df['ville'].str.replace('-', ' ')

In [101]:
df[df['ville'].str.contains('Boulo')]

,id,offre,url_page,ville,pays,entreprise,contrat,date_publi,id_plateforme,description,lat,lon
3,44867047,Lead Data Analyst H/F,https://bouyguestelecom-recrute.talent-soft.co...,Boulogne Billancourt,France,Bouygues Telecom,CDI,2024-02-09,Hello Work,Localisation : Boulogne (92100)\r\nType de con...,48.835412,2.242259
47,1049959,CDI - HR Data Analyst - Pilotage effectifs et ...,https://candidat.pole-emploi.fr/offres/recherc...,Boulogne Billancourt,France,None,CDI,2024-02-08,Pole Emploi,Faisons connaissance\nLa Direction des Service...,48.836976,2.242716
173,17701,CDI - HR Data Analyst - Pilotage effectifs et ...,https://www.talentify.io/job/cdi-hr-data-analy...,Boulogne Billancourt,France,Groupe TF1,Full-Time,2024-02-05,talentify,Faisons connaissance La Direction des Servi...,48.839700,2.239400
178,17558,CDI - Product data analyst - H/F\n,https://www.talentify.io/job/cdi-product-data-...,Boulogne Billancourt,France,Groupe TF1,Full-Time,2024-02-04,talentify,"Faisons connaissance Au sein du Groupe TF1,...",48.839700,2.239400
204,0771521,"Data Analyst F/H - Système, réseaux, données (...",https://candidat.pole-emploi.fr/offres/recherc...,Boulogne Billancourt,France,AI&Data,CDI,2024-02-02,Pole Emploi,Descriptif du poste:\n\nAI&Data recherche auj...,48.836976,2.242716
325,JOBREQ_50163293,CS 24 - Stage BAC+4/5 T - Data Analyst Perform...,https://www.talentify.io/job/cs-24-stage-bac-4...,Boulogne Billancourt,France,Renault Group,Full-Time,2024-01-25,talentify,Company RENAULT s.a.s. Job Description Co...,48.839700,2.239400
332,JOBREQ_50162956,CS 24 - Stage BAC + 5 - Chargée/Chargé RH Dat...,https://www.talentify.io/job/cs-24-stage-bac-5...,Boulogne Billancourt,France,Renault Group,Full-Time,2024-01-25,talentify,Company RENAULT s.a.s. Job Description C...,48.839700,2.239400
383,17512,STAGE - Data Analyst F/H\n,https://www.talentify.io/job/stage-assistant-d...,Boulogne Billancourt,France,Groupe TF1,Full-Time,2024-01-21,talentify,Faisons connaissance En tant que régie publ...,48.839700,2.239400
386,41645119,Data Analyst H/F,https://jobs.smartrecruiters.com/ALTEN/7439999...,Boulogne Billancourt,France,Alten,CDI,2024-01-20,Hello Work,Rejoignez les ingénieurs de la Business Unit P...,48.836640,2.237809
394,17133,Stage - Data Quality Analyst F/H\n,https://www.talentify.io/job/stage-data-qualit...,Boulogne Billancourt,France,Groupe TF1,Full-Time,2024-01-18,talentify,Faisons connaissance Première chaîne de t...,48.839700,2.239400


In [102]:
df['contrat'] = df['contrat'].replace('Full time', 'Full-Time')

In [103]:
df['ville'] = df['ville'].replace('Paris (Dept.)', 'Paris')

In [104]:
df['ville'] = df['ville'].str.title()

In [105]:
df['competences'] = df.description.apply(lambda x : preprocess_text(x))

In [106]:
df = df.drop(df[df['ville'] == ''].index)

In [107]:
competences_tech = [
            'SQL', 'Python', 'R', 'Java', 'C++', 'C#', 'JavaScript',
            'HTML', 'CSS', 'React', 'Angular', 'Vue.js',
            'Data Analysis', 'Data Science', 'Machine Learning', 'Deep Learning',
            'Statistical Analysis', 'Big Data', 'Hadoop', 'Spark',
            'Database Management', 'MySQL', 'PostgreSQL', 'MongoDB', 'SQLite',
            'Data Visualization', 'Tableau', 'Power BI', 'Matplotlib', 'Seaborn',
            'Excel', 'Pandas', 'NumPy', 'SciPy', 'Scikit-learn',
            'ETL', 'Data Warehousing', 'Business Intelligence',
            'Version Control', 'Git', 'GitHub', 'GitLab',
            'APIs', 'RESTful', 'SOAP', 'GraphQL',
            'Web Development', 'Frontend', 'Backend', 'Full Stack',
            'Linux', 'Unix', 'Shell Scripting', 'Bash',
            'Cloud Computing', 'AWS', 'Azure', 'Google Cloud Platform',
            'Docker', 'Kubernetes', 'Containerization',
            'DevOps', 'Continuous Integration', 'Continuous Deployment',
            'Agile', 'Scrum', 'Kanban'
                                    ]

In [108]:
df['competences_matched'] = df['competences'].apply(
                    lambda skills: [skill for skill in skills if skill.lower() in [comp.lower() for comp in competences_tech]])

In [109]:
df[df['ville'].str.contains('Paris')]['ville'].value_counts()

ville
Paris    108
Name: count, dtype: int64

In [110]:
df['ville'].value_counts()

ville
Paris                   108
Lyon                     28
Lille                    16
Issy Les Moulineaux      15
Boulogne Billancourt     12
                       ... 
Eringhem                  1
Poitiers                  1
Cabourg                   1
Eyragues                  1
Carros                    1
Name: count, Length: 147, dtype: int64

In [111]:
df = df.reset_index(drop = True)

In [112]:
df.to_csv('database.csv', index=False)